In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
DATAPATH = "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer/data/"

In [ ]:
import pandas as pd
import os
import numpy as np
import gensim
from gensim.models import Word2Vec, FastText
#import glove
#from glove import Corpus

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, merge, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D, merge

#from keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from keras.utils import np_utils
#from keras.backend.tensorflow_backend import set_session, clear_session, get_session
from keras.backend import set_session, clear_session, get_session
import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:

new_notes = pd.read_pickle(os.path.join(DATAPATH, "ner_df_final_los.p")) # med7
w2vec = Word2Vec.load("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer/embeddings/word2vec.model")
#fasttext = gensim.models.fasttext.load_facebook_model("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer/embeddings/cc.en.300.bin")
fasttext = FastText.load("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer/embeddings/fasttext.model")

In [ ]:
null_index_list = []
for i in new_notes.itertuples():
    
    if len(i.ner) == 0:
    #if not i.ner:
        null_index_list.append(i.Index)
new_notes.drop(null_index_list, inplace=True)
print(len(new_notes))

In [ ]:
med7_ner_data = {}

for ii in new_notes.itertuples():
    
    p_id = ii.SUBJECT_ID
    ind = ii.Index
    
    try:
        new_ner = new_notes.loc[ind].ner
    except:
        new_ner = []
            
    unique = set()
    new_temp = []
    
    for j in new_ner:
        for k in j:
            
            unique.add(k[0])
            new_temp.append(k)

    if p_id in med7_ner_data:
        for i in new_temp:
            med7_ner_data[p_id].append(i)
    else:
        med7_ner_data[p_id] = new_temp
print(len(med7_ner_data))

In [ ]:
pd.to_pickle(med7_ner_data, (os.path.join(DATAPATH, "new_ner_word_dict_los.pkl")))

In [ ]:
def mean(a):
    return sum(a) / len(a)

In [ ]:
data_types = [med7_ner_data]
data_names = ["new_ner"]

for data, names in zip(data_types, data_names):
    new_word2vec = {}
    print("w2vec starting..")
    for k,v in data.items():

        patient_temp = []
        for i in v:
            try:
                patient_temp.append(w2vec[i[0]])
            except:
                avg = []
                num = 0
                temp = []

                if len(i[0].split(" ")) > 1:
                    for each_word in i[0].split(" "):
                        try:
                            temp = w2vec[each_word]
                            avg.append(temp)
                            num += 1
                        except:
                            pass
                    if num == 0: continue
                    avg = np.asarray(avg)
                    #t = np.asarray(map(mean, zip(*avg)))
                    t = np.asarray(list(map(mean, zip(*avg))))
                    patient_temp.append(t)
        if len(patient_temp) == 0: continue
        new_word2vec[k] = patient_temp
    print(len(new_word2vec))
    #############################################################################
    print("fasttext starting..")
        
    new_fasttextvec = {}

    for k,v in data.items():

        patient_temp = []

        for i in v:
            try:
                patient_temp.append(fasttext[i[0]])
            except:
                pass
        if len(patient_temp) == 0: continue
        new_fasttextvec[k] = patient_temp
    print(len(new_fasttextvec))
    #############################################################################    

    print("combined starting..")
    new_concatvec = {}

    for k,v in data.items():
        patient_temp = []
    #     if k != 6: continue
        for i in v:
            w2vec_temp = []
            fasttemp = [] # nitin added 
            try:
                w2vec_temp = w2vec[i[0]]
            except:
                avg = []
                num = 0
                temp = []

                if len(i[0].split(" ")) > 1:
                    for each_word in i[0].split(" "):
                        try:
                            temp = w2vec[each_word]
                            avg.append(temp)
                            num += 1
                        except:
                            pass
                    if num == 0: 
                        #w2vec_temp = [0] * 100
                        w2vec_temp = [0] * 200
                    else:
                        avg = np.asarray(avg)
                        w2vec_temp = np.asarray(list(map(mean, zip(*avg))))  
                else:
                    #w2vec_temp = [0] * 100
                    w2vec_temp = [0] * 200
            #print('value for i',i)    
            try:
                fasttemp = fasttext[i[0]]
            except:
                #fasttemp = [0] * 100
                fasttemp = [0] * 200
            #print('fasttemp {0}, w2vec_temp{1},'.format(fasttemp,w2vec_temp))
            appended = np.append(fasttemp, w2vec_temp, 0)
            patient_temp.append(appended)
        if len(patient_temp) == 0: continue
        new_concatvec[k] = patient_temp



    #print(len(new_word2vec), len(new_fasttextvec))
    print(len(new_word2vec), len(new_fasttextvec), len(new_concatvec))
    
    pd.to_pickle(new_word2vec, (os.path.join(DATAPATH, "new_ner_word2vec_dict_los.pkl")))
    pd.to_pickle(new_fasttextvec, (os.path.join(DATAPATH, "new_ner_fasttext_dict_los.pkl")))
    pd.to_pickle(new_concatvec, (os.path.join(DATAPATH, "new_ner_combined_dict_los.pkl")))


In [ ]:
new_word2vec_dict = pd.read_pickle(os.path.join(DATAPATH, "new_ner_word2vec_dict_los.pkl"))
new_fasttext_dict = pd.read_pickle(os.path.join(DATAPATH, "new_ner_fasttext_dict_los.pkl"))
new_combined_dict = pd.read_pickle(os.path.join(DATAPATH, "new_ner_combined_dict_los.pkl"))

diff = set(new_fasttext_dict.keys()).difference(set(new_word2vec_dict))
for i in diff:
    del new_fasttext_dict[i]
    del new_combined_dict[i]
print (len(new_word2vec_dict), len(new_fasttext_dict), len(new_combined_dict))

pd.to_pickle(new_word2vec_dict, (os.path.join(DATAPATH, "new_ner_word2vec_limited_dict_los.pkl")))
pd.to_pickle(new_fasttext_dict, (os.path.join(DATAPATH, "new_ner_fasttext_limited_dict_los.pkl")))
pd.to_pickle(new_combined_dict, (os.path.join(DATAPATH, "new_ner_combined_limited_dict_los.pkl")))